<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #8: Modelado de temas Jhonathan Giraldo
`Fecha de entrega: 🎃 Octubre 31, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [8]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [9]:
# Cargar datos
path = 'reviews_vidjew_es.csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [10]:
import re
from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

In [11]:
def pre_procesado(texto):
    texto = texto.lower()

    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto
 
data['prep'] = data['review_body'].apply(lambda texto: pre_procesado(texto))
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,prep
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[13 pts]` Punto 2: Modelo de LDA

In [13]:
# Crear una representación de los documentos en forma de diccionario

dictionary = Dictionary(data['prep'].values)

# Filtrar palabras muy frecuentes o infrecuentes

dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in data['prep'].values]

#Train the topic model

model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=7, passes=20)


In [14]:

model.print_topics(num_words=10)#Results


[(0,
  '0.026*"juego" + 0.022*"caja" + 0.021*"mejor" + 0.021*"mando" + 0.020*"si" + 0.019*"mas" + 0.017*"jugar" + 0.015*"así" + 0.014*"después" + 0.013*"mal"'),
 (1,
  '0.045*"calidad" + 0.029*"precio" + 0.020*"puede" + 0.017*"bien" + 0.017*"si" + 0.016*"mala" + 0.015*"pedir" + 0.015*"parece" + 0.013*"pendientes" + 0.012*"español"'),
 (2,
  '0.041*"foto" + 0.038*"llegado" + 0.029*"producto" + 0.022*"bien" + 0.016*"día" + 0.015*"así" + 0.014*"buen" + 0.014*"solo" + 0.013*"amazon" + 0.012*"queda"'),
 (3,
  '0.033*"si" + 0.027*"plata" + 0.026*"bien" + 0.021*"producto" + 0.018*"tiempo" + 0.018*"llego" + 0.017*"llegó" + 0.016*"precio" + 0.014*"perfecto" + 0.013*"bonito"'),
 (4,
  '0.065*"bien" + 0.037*"calidad" + 0.024*"precio" + 0.022*"bastante" + 0.021*"bonitos" + 0.018*"cadena" + 0.018*"buen" + 0.017*"buena" + 0.016*"pequeños" + 0.015*"grandes"'),
 (5,
  '0.037*"buena" + 0.033*"juego" + 0.026*"calidad" + 0.025*"precio" + 0.024*"perfecto" + 0.021*"entrega" + 0.021*"buen" + 0.021*"regalo" 

###  `[25 pts]` Punto 3: Visualización de LDA

In [15]:
plot_modelo = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(plot_modelo)